In [ ]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from pathlib import Path
import shutil
import re
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

###Section 2

In [ ]:
!unzip '/content/drive/MyDrive/archive (2).zip'

Archive:  /content/drive/MyDrive/archive (2).zip
  inflating: tweets.csv              


In [ ]:
data_path = Path('/content')
tweet_data_path = data_path / 'tweets.csv'

tweet_data = pd.read_csv(tweet_data_path,
                         sep=';' ,
                          usecols=['timestamp','text'],
                          on_bad_lines='skip')

<ipython-input-4-56c0db82a36e>:4: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet_data = pd.read_csv(tweet_data_path,


In [ ]:
tweet_data['text'] = tweet_data['text'].astype(str)
tweet_data['timestamp'] = pd.to_datetime(tweet_data['timestamp'])
min(tweet_data['timestamp']),max(tweet_data['timestamp'])

(Timestamp('2007-04-19 07:14:38+0000', tz='UTC'),
 Timestamp('2019-11-23 15:45:57+0000', tz='UTC'))

In [ ]:
price = pd.read_csv('/content/BTC_USD.csv')
price['Date'] = pd.to_datetime(price['Date'])

min_date = min(price['Date']).tz_localize('UTC')
max_date = max(price['Date']).tz_localize('UTC')
print(f'{min_date}\n{max_date}')

tweet_data = tweet_data[(tweet_data['timestamp'] >= min_date) & (tweet_data['timestamp'] <= max_date) ]


(Timestamp('2015-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2025-01-26 00:00:00+0000', tz='UTC'))

In [ ]:
path = Path('/content/drive/MyDrive/Bitcoin_data')
tweet_data.to_csv(path/'tweets_2015_2019.csv',index=False)

###Section 3

In [ ]:
data_path = Path('/content/drive/MyDrive/Bitcoin_data')
tweet_data_path = data_path / 'tweets_2015_2019_cleaned (1).csv'

tweet_data = pd.read_csv('/content/drive/MyDrive/Bitcoin_data/tweets_2015_2019_cleaned (1).csv')

In [ ]:
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # Emoticons
    u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & map symbols
    u"\U0001F700-\U0001F77F"  # Alchemical symbols
    u"\U0001F780-\U0001F7FF"  # Geometric shapes
    u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    u"\U0001FA00-\U0001FA6F"  # Chess symbols
    u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    u"\U00002702-\U000027B0"  # Dingbats
    u"\U000024C2-\U0001F251"  # Enclosed characters
    "]+", flags=re.UNICODE
)

emoticon_pattern = re.compile(
    r"(:-\)|:\)|:-\(|:\(|:-D|:D|;-D|;-P|;P|:-P|:P|:-O|:O|:-/|:/|:-\||:\||:-\*|:\*)"
)



tweet_data['text'] = tweet_data['text'].str.replace(emoji_pattern, ' ', regex=True) # Replace Emojis with an Empty String
tweet_data['text'] = tweet_data['text'].str.replace(emoticon_pattern, ' ', regex=True)
tweet_data['text'] = tweet_data['text'].str.replace('https?\.?\S+',' ',regex=True) # Removing the url from the text
tweet_data['text'] = tweet_data['text'].str.replace('#',' ',regex=True) # removing # from the data
tweet_data['text'] = tweet_data['text'].str.replace('RT',' ',regex=True) # Removing the Re-tweet mark
tweet_data['text'] = tweet_data['text'].str.replace('@[\w]',' ',regex=True) #removing the twitter handles @user
tweet_data['text'] = tweet_data['text'].str.lower()


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_pattern = r'\b(?:' + '|'.join(map(re.escape, stop_words)) + r')\b'
tweet_data['text'] = (
    tweet_data['text']
    .str.replace(stop_pattern, ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)  # Remove extra spaces
    .str.strip()  # Trim leading/trailing spaces
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tweet_data.to_csv('/content/drive/MyDrive/Bitcoin_data/tweets_2015_2019_cleaned.csv', index=False)

###Section 4


In [ ]:
data_path = Path('/content/drive/MyDrive/Bitcoin_data')
tweet_data_path = data_path / 'tweets_2015_2019_cleaned.csv'
tweet_data = pd.read_csv(tweet_data_path)

In [ ]:
model_ckpt = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassi

In [ ]:
all_ =  []
count = 0
for i in tqdm(range( len(tweet_data['text']))):
  try:
      encoded_input = tokenizer(tweet_data['text'][i],
                                return_token_type_ids=False,  # This tokenizer does not take token_type_ids
                                padding=True,  # Pad the sequences to the same length
                                truncation=True,  # Truncate sequences longer than the model's max input length
                                return_tensors='tf',  # Return TensorFlow tensors
                                max_length=64)
  except:
      all_.append(i)
      continue

np.save('/content/drive/MyDrive/state.npy',np.array(all_))
tweet_data.drop(index=all_, axis=0, inplace=True)
tweet_data.to_csv('/content/drive/MyDrive/tweets_2015_2019_cleaned.csv', index=False)
len(all_)

  0%|          | 0/18363196 [00:00<?, ?it/s]

In [ ]:
lan_path = '/content/drive/MyDrive/lan.npy'
batch_size = 3100
lans = []
c = 0
for i in tqdm(range(18175224, len(tweet_data), batch_size)):
    encoded_input = tokenizer(tweet_data['text'][i:].to_list(),
                                return_token_type_ids=False,  # This tokenizer does not take token_type_ids
                                padding=True,  # Pad the sequences to the same length
                                truncation=True,  # Truncate sequences longer than the model's max input length
                                return_tensors='tf',  # Return TensorFlow tensors
                                max_length=64)

    lan = np.argmax(model(**encoded_input)[0].numpy(),axis=1)
    lans.append(lan)
    c += 1
    if  100:
      np.save(lan_path,np.concatenate(lans))
      print(f'lan {i} is saved ')
      c=0


In [ ]:
model.config.id2label

{0: 'ja',
 1: 'nl',
 2: 'ar',
 3: 'pl',
 4: 'de',
 5: 'it',
 6: 'pt',
 7: 'tr',
 8: 'es',
 9: 'hi',
 10: 'el',
 11: 'ur',
 12: 'bg',
 13: 'en',
 14: 'fr',
 15: 'zh',
 16: 'ru',
 17: 'th',
 18: 'sw',
 19: 'vi'}

In [ ]:
tweet_data = twreet_data[np.concatenate(lans)==13]

In [ ]:
tweet_data_path = '/content/drive/MyDrive/tweets_2015_2019_cleaned_EN.csv'
tweet_data = pd.read_csv(tweet_data_path)